In [1]:
import pandas as pd
import numpy as np
from api_utils import init_settings

start_date,end_date = init_settings()

In [2]:
df = pd.read_csv(f'data/01_raw/{start_date}_{end_date}.csv')
df.head()

,date,lat,long,country,city,mintemp_c,maxtemp_c,avgtemp_c,avghumidity,totalprecip_mm,description
0,2023-01-01,-34.59,-58.67,Argentina,Buenos Aires,21.1,27.0,24.2,52.0,8.7,Moderate or heavy rain shower
1,2023-01-02,-34.59,-58.67,Argentina,Buenos Aires,16.3,33.2,24.5,45.0,0.0,Sunny
2,2023-01-03,-34.59,-58.67,Argentina,Buenos Aires,18.8,33.5,26.7,26.0,0.1,Patchy rain possible
3,2023-01-04,-34.59,-58.67,Argentina,Buenos Aires,18.3,31.9,25.0,42.0,0.0,Sunny
4,2023-01-05,-34.59,-58.67,Argentina,Buenos Aires,19.2,34.0,26.3,42.0,0.0,Sunny


In [3]:
df.describe()

,lat,long,mintemp_c,maxtemp_c,avgtemp_c,avghumidity,totalprecip_mm
count,1.340000e+02,1.340000e+02,134.000000,134.000000,134.000000,134.000000,134.000000
mean,-3.459000e+01,-5.867000e+01,18.846269,30.417910,23.954478,53.365672,2.349701
std,6.418881e-14,5.705672e-14,4.789096,6.340037,5.227682,14.452692,5.699239
min,-3.459000e+01,-5.867000e+01,9.500000,16.000000,14.000000,24.000000,0.000000
25%,-3.459000e+01,-5.867000e+01,14.950000,24.925000,19.675000,43.250000,0.000000
50%,-3.459000e+01,-5.867000e+01,19.200000,31.200000,24.650000,53.000000,0.000000
75%,-3.459000e+01,-5.867000e+01,22.500000,35.175000,28.100000,63.750000,0.675000
max,-3.459000e+01,-5.867000e+01,29.500000,43.900000,35.100000,94.000000,31.300000


### *Transformations*:
#### Deribed columns
1. *cloud_coverage*: [boolean] there was a cloudy sky/ low visibility
2. *rain*: [boolean]  it rained
3. *rain_tomorrow*: [boolean] next day rained
4. *mean_avg_humidity_3days*: [float] 3 days window mean of avgtemp_c (media movil)
5. *humidity_ratio*: [float] daily humidity encrease vs previous day

#### 1. cloud_coverage

In [4]:
df["description"].value_counts()

Sunny                             51
Partly cloudy                     21
Patchy rain possible              18
Moderate or heavy rain shower      8
Overcast                           8
Cloudy                             7
Light rain shower                  7
Moderate rain at times             5
Moderate rain                      2
Patchy light rain with thunder     1
Patchy light rain                  1
Light drizzle                      1
Fog                                1
Light rain                         1
Heavy rain                         1
Mist                               1
Name: description, dtype: int64

In [5]:
df["description"] = df["description"].str.lower()
df["cloud_coverage"]=np.where(df["description"].str.contains("sunny"), 0, 1)


In [6]:
df["cloud_coverage"].value_counts()

1    83
0    51
Name: cloud_coverage, dtype: int64

#### 2. rain

In [7]:
df["rain"] = np.where(df["totalprecip_mm"]>0,1,0)
df["rain"].value_counts()

0    86
1    48
Name: rain, dtype: int64

#### 3. rain_tomorrow

In [8]:
df['rain_tomorrow'] = df['rain'].shift(-1)
df[['rain_tomorrow','rain',"date"]].head()

,rain_tomorrow,rain,date
0,0.0,1,2023-01-01
1,1.0,0,2023-01-02
2,0.0,1,2023-01-03
3,0.0,0,2023-01-04
4,0.0,0,2023-01-05


#### 4. mean_avg_humidity_3days

In [9]:
def calculate_mean_avg_humidity(data):
    # Sort the data by date in ascending order
    data.sort_values('date', inplace=True)
    
    # Create a new column to store the mean of the previous 3 days' average humidity
    data['mean_avghumidity_3days'] = pd.Series(dtype='float64')
    
    # Iterate over each row in the dataframe
    for i, row in data.iterrows():
        if i >= 3:
            # Calculate the mean of the previous 3 days' average humidity
            mean_avg_humidity_3days = data.loc[i-3:i-1, 'avghumidity'].mean()
            data.at[i, 'mean_avghumidity_3days'] = mean_avg_humidity_3days
    
    return data['mean_avghumidity_3days']
    
df['mean_avghumidity_3days'] = calculate_mean_avg_humidity(df)
df[["date","avghumidity","mean_avghumidity_3days"]].head()

,date,avghumidity,mean_avghumidity_3days
0,2023-01-01,52.0,NaN
1,2023-01-02,45.0,NaN
2,2023-01-03,26.0,NaN
3,2023-01-04,42.0,41.000000
4,2023-01-05,42.0,37.666667


#### 5. humidity_ratio

In [10]:
def calculate_humidity_ratio(data):
    # Sort the data by date in ascending order
    data.sort_values('date', inplace=True)
    
    # Calculate the ratio of avg_humidity to the previous day's avg_humidity
    return data.loc[:, 'avghumidity'].pct_change().round(decimals=2)
    
df['humidity_ratio'] = calculate_humidity_ratio(df)
df.head()

,date,lat,long,country,city,mintemp_c,maxtemp_c,avgtemp_c,avghumidity,totalprecip_mm,description,cloud_coverage,rain,rain_tomorrow,mean_avghumidity_3days,humidity_ratio
0,2023-01-01,-34.59,-58.67,Argentina,Buenos Aires,21.1,27.0,24.2,52.0,8.7,moderate or heavy rain shower,1,1,0.0,NaN,NaN
1,2023-01-02,-34.59,-58.67,Argentina,Buenos Aires,16.3,33.2,24.5,45.0,0.0,sunny,0,0,1.0,NaN,-0.13
2,2023-01-03,-34.59,-58.67,Argentina,Buenos Aires,18.8,33.5,26.7,26.0,0.1,patchy rain possible,1,1,0.0,NaN,-0.42
3,2023-01-04,-34.59,-58.67,Argentina,Buenos Aires,18.3,31.9,25.0,42.0,0.0,sunny,0,0,0.0,41.000000,0.62
4,2023-01-05,-34.59,-58.67,Argentina,Buenos Aires,19.2,34.0,26.3,42.0,0.0,sunny,0,0,0.0,37.666667,0.00


In [11]:
df.head()

,date,lat,long,country,city,mintemp_c,maxtemp_c,avgtemp_c,avghumidity,totalprecip_mm,description,cloud_coverage,rain,rain_tomorrow,mean_avghumidity_3days,humidity_ratio
0,2023-01-01,-34.59,-58.67,Argentina,Buenos Aires,21.1,27.0,24.2,52.0,8.7,moderate or heavy rain shower,1,1,0.0,NaN,NaN
1,2023-01-02,-34.59,-58.67,Argentina,Buenos Aires,16.3,33.2,24.5,45.0,0.0,sunny,0,0,1.0,NaN,-0.13
2,2023-01-03,-34.59,-58.67,Argentina,Buenos Aires,18.8,33.5,26.7,26.0,0.1,patchy rain possible,1,1,0.0,NaN,-0.42
3,2023-01-04,-34.59,-58.67,Argentina,Buenos Aires,18.3,31.9,25.0,42.0,0.0,sunny,0,0,0.0,41.000000,0.62
4,2023-01-05,-34.59,-58.67,Argentina,Buenos Aires,19.2,34.0,26.3,42.0,0.0,sunny,0,0,0.0,37.666667,0.00


#### Store transformed data

In [12]:
df.to_csv(f'data/02_transformed/{start_date}_{end_date}.csv',index=False)